In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from fairlearn.metrics import MetricFrame, selection_rate, demographic_parity_difference, equalized_odds_difference
from fairlearn.postprocessing import ThresholdOptimizer

In [5]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
data = pd.read_csv(url, names=column_names, sep=",\s*", engine="python")

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\rajro\AppData\Local\Temp\ipykernel_14408\2602640992.py:7: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_csv(url, names=column_names, sep=",\s*", engine="python")


In [ ]:
data.replace('?', np.nan, inplace=True)
data.dropna(inplace=True)

# Label encode categorical features
categorical_cols = data.select_dtypes(include='object').columns
le = LabelEncoder()
for col in categorical_cols:
    data[col] = le.fit_transform(data[col])

X = data.drop("income", axis=1)
y = data["income"]  


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)


clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))

Accuracy: 0.8172173720853133
Recall: 0.4480461811722913


In [8]:
# Step 3: Fairness Audit
sensitive_feature_name = "sex"
sensitive_features = X.iloc[X_test.argmax(axis=1)][sensitive_feature_name].values

metrics = {
    "accuracy": accuracy_score,
    "recall": recall_score,
    "selection_rate": selection_rate
}

frame = MetricFrame(metrics=metrics, y_true=y_test, y_pred=y_pred, sensitive_features=sensitive_features)
print("\nFairness Metrics by Group (Sex):")
print(frame.by_group)

# Fairness differences
print("\nDemographic Parity Difference:", demographic_parity_difference(y_test, y_pred, sensitive_features=sensitive_features))
print("Equalized Odds Difference:", equalized_odds_difference(y_test, y_pred, sensitive_features=sensitive_features))

# Step 4: Apply Bias Mitigation (Post-processing)
clf = LogisticRegression(solver='liblinear')
clf.fit(X_train, y_train)

thresh_opt = ThresholdOptimizer(
    estimator=clf,
    constraints="demographic_parity",
    prefit=True
)

X_test_df = pd.DataFrame(X_test, columns=X.columns)
X_test_df_sensitive = X.iloc[X_test.argmax(axis=1)][sensitive_feature_name].values

thresh_opt.fit(X_test_df, y_test, sensitive_features=X_test_df_sensitive)
y_pred_fair = thresh_opt.predict(X_test_df, sensitive_features=X_test_df_sensitive)

# Evaluate mitigated predictions
fair_frame = MetricFrame(metrics=metrics, y_true=y_test, y_pred=y_pred_fair, sensitive_features=X_test_df_sensitive)
print("\nPost-Mitigation Fairness Metrics by Group (Sex):")
print(fair_frame.by_group)


Fairness Metrics by Group (Sex):
                     accuracy    recall  selection_rate
sensitive_feature_0                                    
0                    0.790950  0.470776        0.192459
1                    0.832403  0.431732        0.136379

Demographic Parity Difference: 0.05607903111339785
Equalized Odds Difference: 0.0390442677872429


c:\Users\rajro\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\Users\rajro\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(



Post-Mitigation Fairness Metrics by Group (Sex):
                     accuracy    recall  selection_rate
sensitive_feature_0                                    
0                    0.787330  0.399575        0.155656
1                    0.832054  0.469108        0.153819
